# CA1286

In [143]:
import pandas as pd
import altair as alt


alt.theme.enable("latimes")

df = pd.read_csv(
    "..\\CodeAnalysis\\BenchmarkDotNet.Artifacts\\results\\Benchmark.Ca1826-report.csv",
    delimiter=";",
)

def clean_data(df):
    # Replace all instances of " ns" with "" in column: 'Mean'
    df['Mean'] = df['Mean'].str.replace(" ns", "", case=False, regex=False)
    # Replace all instances of "," with "" in column: 'Mean'
    df['Mean'] = df['Mean'].str.replace(",", "", case=False, regex=False)
    # Change column type to float64 for column: 'Mean'
    df = df.astype({'Mean': 'float64'})
    # Convert to microsecond
    df['Mean'] = df['Mean'] / 1000
    # Replace all instances of "Linq" with "" in column: 'Method'
    df['Method'] = df['Method'].str.replace("Linq", "", case=False, regex=False)
    # Replace all instances of "Prop" with "" in column: 'Method'
    df['Method'] = df['Method'].str.replace("Prop", "", case=False, regex=False)
    return df

df_clean = clean_data(df.copy())
df_clean.head()

plot = (
    alt.Chart(df_clean)
    .mark_bar()
    .encode(
        y=alt.Y("Method").title(""),
        x=alt.X("Mean").title("Execution time, μs"),
        color="Categories",
        text="Method",
        row="Categories"
    )
    .properties(
        title={
            "text": "LINQ methods vs direct property access performance",
            "subtitle": "For IReadOnlyList<string> of length 1000",
        },
        height=200,
        width=450,
    )
)

plot.show()
plot.save("results.png", scale_factor=2)

alt.Chart(...)